# Example 1: Kalman Filter

## Contents
* [Acknowledgements](#ackw)
* [Overview](#overview) 
    * [Kalman Filter](#kf)
    * [Summary Kalman Filter](#sumekf)
    * [Motion Model](#motion_model)
    * [Observation Model](#observation_model)
* [Include files](#include_files)
* [The main function](#m_func)
* [Results](#results)
* [Source Code](#source_code)

## <a name="overview"></a> Overview

### <a name="kf"></a> Kalman Filter

### <a name="sumekf"></a> Summary of Kalman Filter

**Prediction Step:**

$$ \hat{\mathbf{x}}_{k} = F_k \mathbf{x}_{k-1} + B_k   \mathbf{u}_k + \mathbf{w}_k$$
    

$$\hat{\mathbf{P}}_{k} = \mathbf{F}_{k-1}  \mathbf{P}_{k-1}  \mathbf{F}_{k-1}^T +  \mathbf{Q}_{k-1}$$

**Update Step:**

$$\mathbf{K}_k = \hat{\mathbf{P}}_{k}  \mathbf{H}_{k}^T  (\mathbf{H}_k  \hat{\mathbf{P}}_{k}  \mathbf{H}_{k}^T +  \mathbf{R}_k )^{-1}$$

$$\mathbf{x}_k = \hat{\mathbf{x}}_{k} + \mathbf{K}_k  (\mathbf{z}_k - \mathbf{H}_k  \hat{\mathbf{x}}_{k})$$

$$\mathbf{P}_k = (\mathbf{I} - \mathbf{K}_k  \mathbf{H}_k)  \hat{\mathbf{P}}_{k}$$

### <a name="motion_model"></a> Motion Model

### <a name="observation_model"></a> Observation Model

## <a name="include_files"></a> Include files

```
#include "cubic_engine/base/cubic_engine_types.h"
#include "cubic_engine/estimation/kalman_filter.h"
#include "kernel/dynamics/motion_model_base.h"
#include "kernel/dynamics/dynamics_matrix_descriptor.h"
#include "kernel/dynamics/system_state.h"
#include "kernel/maths/statistics/multivariate_normal_distribution.h"
#include "kernel/utilities/csv_file_writer.h"

#include <iostream>
#include <random>
```

## <a name="m_func"></a> The main function

```
namespace example
{

using cengine::real_t;
using cengine::uint_t;
using cengine::DynMat;
using cengine::DynVec;
using cengine::DiagMat;
using cengine::KalmanFilter;
using kernel::dynamics::MotionModelBase;
using kernel::dynamics::DynamicsMatrixDescriptor;
using kernel::dynamics::SysState;
using kernel::maths::stats::MultiNormalDist;
using kernel::CSVWriter;

const uint_t N_ITRS = 1000;
const real_t DT = 0.5;
const real_t U0 = -2.0;

class MotionModel: public MotionModelBase<SysState<2>,
                                          DynamicsMatrixDescriptor, DynVec<real_t>>
{

public:

    typedef MotionModelBase<SysState<2>,
                            DynamicsMatrixDescriptor, DynVec<real_t>>::input_t input_t;

    typedef MotionModelBase<SysState<2>,
                            DynamicsMatrixDescriptor, DynVec<real_t>>::state_t state_t;


    /// Updates and returns the value of the state given the input
    virtual state_t& evaluate(const input_t& /*input*/)override final
    {throw std::logic_error("Not Implemented");}

private:
};


class ObservationModel
{

public:

    typedef real_t input_t;

    /// constructor
    ObservationModel();

    /// Observe
    real_t evaluate(const DynVec<real_t>& input);

    /// brief Return the matrix
    const DynMat<real_t>& get_matrix(const std::string& /*name*/)const{
        return H_;
    }

private:

    /// the white noise for the observation
    std::normal_distribution<real_t> distribution_;

    /// The matrix H
    DynMat<real_t> H_;

};

ObservationModel::ObservationModel()
    :
    distribution_ (0.0,0.05),
    H_(1,2,0.0)
{
    H_(0,0) = 1.0;
    H_(0,1) = 0.0;
}

real_t
ObservationModel::evaluate(const DynVec<real_t>& input){

    std::random_device rd;
    std::mt19937 gen(rd());
    auto error = distribution_(gen);
    return 2.2; //(H_ * input)[0] + error;

}


}

int main() {

    using namespace example;

    std::cout<<"Running Example"<<std::endl;

    MotionModel motion_model;
    DynMat<real_t> F(2, 2, 0.0);
    F(0,0) = 1.0;
    F(0, 1) = DT;
    F(1, 0) = 0.0;
    F(1, 1) = 1.0;

    motion_model.set_matrix("F", F);

    /// the object that handles the error distribution
    MultiNormalDist dist_error(DynVec<real_t>(2, 0.0), DiagMat<real_t>(2,0.1) );

    DynVec<real_t> mu_init(2, 0.0);
    mu_init[1] = 5.0;

    DiagMat<real_t> sigma_init(2,0.01);
    sigma_init(1,1)=1;

    /// the initial data
    MultiNormalDist init_data(mu_init, sigma_init );

    /// the initial state
    auto state_0 = init_data.sample();

    motion_model.set_state_name_value(0, "P", state_0[0]);
    motion_model.set_state_name_value(1, "V", state_0[1]);

    ObservationModel obs_model;

    KalmanFilter<MotionModel, ObservationModel> kf(motion_model, obs_model);

    DynMat<real_t> P(2, 2, 0.0);
    P(0, 0) = 1.0;
    P(1, 1) = 1.0;

    kf.set_matrix("P", P);

    DynMat<real_t> Q(2, 2, 0.0);
    Q(0, 0) = 0.1;
    Q(1, 1) = 0.1;

    kf.set_matrix("Q", Q);

    DynMat<real_t> B(2, 1, 0.0);
    B(0, 0) = 0.0;
    B(1, 0) = DT;
    kf.set_matrix("B", B);

    DynMat<real_t> R(1, 1, 0.05);
    kf.set_matrix("R", R);

    CSVWriter writer("state.csv", ',', true);
    std::vector<std::string> names{"Time", "P", "V"};
    writer.write_column_names(names);
    real_t time = 0.0;

    for(uint_t itr=0; itr<N_ITRS; ++itr){

        std::cout<<"At time: "<<time<<std::endl;
        auto& state = kf.get_state();

        std::cout<<"State: "<<state.as_string()<<std::endl;

        auto obs = obs_model.evaluate(state.as_vector());
        DynVec<real_t> error = dist_error.sample();
        kf.estimate(std::make_tuple(DynVec<real_t>(1,U0), error, obs));

        writer.write_row(std::make_tuple(time, state.get("P"), state.get("V")));

        time += DT;
    }
    
    return 0;
}

```

## <a name="results"></a> Results

```
Running Example
At time: 0
State: P:0.215814,V:6.660757,
At time: 0.5
State: P:2.241234,V:5.248414,
At time: 1
State: P:2.454916,V:1.808499,
At time: 1.5
State: P:2.272105,V:0.364620,
At time: 2
State: P:2.251736,V:-0.861922,
At time: 2.5
State: P:2.127780,V:-1.590895,
At time: 3
State: P:2.034480,V:-2.006121,
At time: 3.5
State: P:2.010868,V:-2.352556,
At time: 4
State: P:1.990048,V:-2.632724,
At time: 4.5
State: P:1.849978,V:-2.435943,
At time: 5
State: P:1.834766,V:-2.188333,
At time: 5.5
State: P:1.905250,V:-2.181821,
At time: 6
State: P:2.076366,V:-2.759683,
At time: 6.5
State: P:1.940992,V:-2.875358,
At time: 7
State: P:2.041201,V:-3.333179,
At time: 7.5
State: P:1.647069,V:-2.445351,
At time: 8
State: P:1.962734,V:-2.635273,
At time: 8.5
State: P:1.961377,V:-2.820562,
At time: 9
State: P:1.889299,V:-2.759763,
At time: 9.5
State: P:1.937315,V:-2.862902,
At time: 10
State: P:1.794920,V:-2.479872,
At time: 10.5
State: P:1.961276,V:-2.664818,
At time: 11
State: P:1.889018,V:-2.603060,
At time: 11.5
State: P:1.990974,V:-2.889402,
At time: 12
State: P:1.889364,V:-2.828824,
At time: 12.5
State: P:1.801871,V:-2.469528,
At time: 13
State: P:1.947506,V:-2.607460,
At time: 13.5
State: P:2.040997,V:-3.064588,
At time: 14
State: P:1.915349,V:-3.092727,
At time: 14.5
State: P:1.936567,V:-3.193309,
At time: 15
State: P:1.895654,V:-3.154206,
At time: 15.5
State: P:1.737708,V:-2.575843,
At time: 16
State: P:1.991729,V:-2.864762,
At time: 16.5
State: P:2.062718,V:-3.396053,
At time: 17
State: P:1.951227,V:-3.546688,
At time: 17.5
State: P:1.787524,V:-3.138407,
At time: 18
State: P:1.869159,V:-3.008847,
At time: 18.5
State: P:1.733467,V:-2.416003,
At time: 19
State: P:1.899036,V:-2.388449,
At time: 19.5
State: P:2.078601,V:-2.973967,
At time: 20
State: P:1.957949,V:-3.147552,
At time: 20.5
State: P:1.903696,V:-3.135908,
At time: 21
State: P:1.870398,V:-3.010575,
At time: 21.5
State: P:1.831857,V:-2.753655,
At time: 22
...

```

<img src="pos_vel.png"
     alt="Position/Velocity vs Time"
     style="float: left; margin-right: 10px;" />

## <a name="source_code"></a> Source Code

<a href="../exe.cpp">exe.cpp</a>